本文将详细讲解 Multi-Head Attention (MHA)、Multi-Query Attention (MQA)、Grouped-Query Attention (GQA) 概念、原理、算法和基础的 Python NumPy 实现。


---

### 总结比较

| 特性           | MHA (Multi-Head Attention)       | MQA (Multi-Query Attention)      | GQA (Grouped-Query Attention)      |
| :------------- | :------------------------------- | :------------------------------- | :--------------------------------- |
| K/V 共享       | 不共享 (每个头独立 K/V)          | 所有头共享同一组 K/V             | 按组共享 (每组共享一组 K/V)        |
| K/V 投影次数   | $h$ 次 (每个头一次)              | 1 次                             | $G$ 次 (每组一次)                  |
| K/V 内存/计算  | 最高 ($O(h \cdot L' \cdot d_k)$) | 最低 ($O(L' \cdot d_k)$)         | 中等 ($O(G \cdot L' \cdot d_k)$)   |
| 推理速度       | 较慢                             | 最快                             | 较快 (介于 MQA 和 MHA 之间)        |
| 模型性能 (通常) | 最好                             | 可能略有下降 (特别是小模型)      | 接近 MHA，优于 MQA (通常)          |
| 实现复杂度     | 标准                             | 略有修改 (K/V 广播)              | 需要按组投影和重复 (略复杂)        |

**主要优势：**

* **MHA:** 通常能提供最佳的模型性能，因为每个头可以学习不同的注意力模式。
* **MQA:** 显著减少 K/V 缓存和计算，极大地提高推理速度和内存效率，尤其适用于长序列生成。
* **GQA:** 在推理效率和模型性能之间取得平衡，通常在性能上比 MQA 更接近 MHA，同时仍提供显著的效率提升。

在大型语言模型（LLMs）的推理阶段，K/V 缓存是主要的内存瓶颈之一。

MQA 和 GQA 通过减少 K/V 的存储需求，成为优化 LLM 推理的关键技术。

MQA 提供了最大的效率提升，而 GQA 则在保持大部分 MHA 性能的同时，提供了良好的效率。

以下是会先构建一些辅助函数，然后介绍具体内容。

首先，我们需要一些通用的辅助函数：

In [1]:
import numpy as np
import math

def softmax(x, axis=-1):
    """Compute softmax values for each sets of scores in x."""
    # For numerical stability
    x = x - np.max(x, axis=axis, keepdims=True)
    e_x = np.exp(x)
    return e_x / np.sum(e_x, axis=axis, keepdims=True)

def scaled_dot_product_attention(q, k, v, mask=None):
    """
    计算缩放点积注意力。

    Args:
        q: Query 张量，形状 (..., seq_len_q, depth)。
        k: Key 张量，形状 (..., seq_len_k, depth)。
        v: Value 张量，形状 (..., seq_len_v, depth_v)。
            seq_len_k 必须等于 seq_len_v。
        mask: Mask 张量 (用于屏蔽某些连接)，形状 (..., seq_len_q, seq_len_k)。
              通常在解码器中使用 (Look-ahead mask) 或处理变长序列 (Padding mask)。

    Returns:
        output: 注意力计算结果，形状 (..., seq_len_q, depth_v)。
        attention_weights: 注意力权重，形状 (..., seq_len_q, seq_len_k)。
    """
    matmul_qk = np.matmul(q, k.transpose(0, 1, 3, 2)) # (..., seq_len_q, seq_len_k)

    d_k = q.shape[-1]
    scaled_attention_logits = matmul_qk / np.sqrt(d_k)

    if mask is not None:
        # 将 mask 为 0 的位置 (即需要忽略的位置) 的 logits 设置为一个非常小的负数
        scaled_attention_logits = scaled_attention_logits + (mask * -1e9)

    attention_weights = softmax(scaled_attention_logits, axis=-1) # (..., seq_len_q, seq_len_k)
    output = np.matmul(attention_weights, v) # (..., seq_len_q, depth_v)

    return output, attention_weights

def split_heads(x, num_heads, depth):
    """
    将最后一个维度分割成 (num_heads, depth)。
    转置结果以便进行 attention 计算。

    Args:
        x: 输入张量，形状 (batch_size, seq_len, d_model)。
        num_heads: 头数。
        depth: 每个头的维度 (d_model / num_heads)。

    Returns:
        一个形状为 (batch_size, num_heads, seq_len, depth) 的张量。
    """
    x = x.reshape(x.shape[0], x.shape[1], num_heads, depth)
    return x.transpose(0, 2, 1, 3)

def combine_heads(x):
    """
    合并多头注意力的输出。

    Args:
        x: 输入张量，形状 (batch_size, num_heads, seq_len, depth)。

    Returns:
        一个形状为 (batch_size, seq_len, d_model) 的张量。
    """
    x = x.transpose(0, 2, 1, 3)
    d_model = x.shape[2] * x.shape[3]
    return x.reshape(x.shape[0], x.shape[1], d_model)

# Helper for initializing dummy weights
def init_weights(*shape):
    return np.random.randn(*shape).astype(np.float32) * 0.01 # Use small random values

# Helper for creating a padding mask (example)
def create_padding_mask(seqs, pad_token=0):
    """
    创建用于注意力机制的填充掩码。
    Args:
        seqs: 输入序列张量 (batch_size, seq_len)。
        pad_token: 填充 token 的 ID。
    Returns:
        一个形状为 (batch_size, 1, 1, seq_len_k) 的掩码张量，填充位置为 1，否则为 0。
    """
    mask = (seqs == pad_token)[:, np.newaxis, np.newaxis, :]
    return mask.astype(np.float32)

# Helper for creating a look-ahead mask (example for decoder self-attention)
def create_look_ahead_mask(seq_len):
    """
    创建用于解码器自注意力的前瞻掩码。
    Args:
        seq_len: 序列长度。
    Returns:
        一个形状为 (1, 1, seq_len, seq_len) 的掩码张量，需要屏蔽的位置为 1，否则为 0。
    """
    mask = 1 - np.tril(np.ones((seq_len, seq_len), dtype=np.float32))
    return mask[np.newaxis, np.newaxis, :, :].astype(np.float32)

现在我们来分别实现和解释各个注意力机制。

---

### 1. MHA (Multi-Head Attention)

* **全文 (Full Name):** Multi-Head Attention (多头注意力)
* **解释 (Explanation):** MHA 是 Transformer 论文中提出的标准注意力机制。它将 Query (Q)、Key (K) 和 Value (V) 分别通过不同的线性投影分成多组（即多个“头”），然后对每个头独立并行地执行缩放点积注意力。最后，将所有头的输出拼接起来，再通过一个最终的线性投影得到最终结果。
* **数学原理 (Mathematical Principle):**
    $$\text{MHA}(Q, K, V) = \text{Concat}(\text{head}_1, ..., \text{head}_h)W^O$$
    其中 $\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)$。
    这里的 $W_i^Q, W_i^K, W_i^V$ 是第 $i$ 个头的线性投影权重矩阵。实际上，通常是使用一个大的权重矩阵 $W^Q, W^K, W^V$ (形状为 $(d_{model}, h \cdot d_k)$ 或 $(d_{model}, d_{model})$ 并后续 reshape) 对 Q, K, V 进行一次性投影，然后将结果在最后一个维度上分割成 $h$ 个头。每个头的维度是 $d_k = d_{model} / h$ (对于 Q 和 K) 和 $d_v = d_{model} / h$ (对于 V)。$W^O$ 是最终的线性输出投影矩阵 (形状为 $(h \cdot d_v, d_{model})$ 或 $(d_{model}, d_{model})$)。
    注意力函数 $\text{Attention}$ 采用缩放点积注意力：
    $$\text{Attention}(Q', K', V') = \text{softmax}(\frac{Q'(K')^T}{\sqrt{d_k}})V'$$
* **算法 (Algorithm Steps):**
    1.  输入 $Q, K, V$ (形状通常为 $(B, L, d_{model})$)。
    2.  对 $Q, K, V$ 应用独立的线性变换（投影）：$Q_{proj} = QW^Q, K_{proj} = KW^K, V_{proj} = VW^V$。这里的权重 $W^Q, W^K, W^V$ 形状为 $(d_{model}, d_{model})$。
    3.  将投影后的 $Q_{proj}, K_{proj}, V_{proj}$ reshape 并转置，分割成 $h$ 个头。形状变为 $(B, h, L, d_k)$。
    4.  在每个头（跨批次和头维度并行）上执行缩放点积注意力计算。
    5.  将所有头的注意力输出结果在头维度上拼接起来，reshape 回原始形状 $(B, L, d_{model})$。
    6.  对拼接后的结果应用最终的线性变换（投影）：$Output = \text{Concat}(\text{heads})W^O$。权重 $W^O$ 形状为 $(d_{model}, d_{model})$。
* **Python 代码实现与测试 (Python Code Implementation & Test):**

In [2]:
def multi_head_attention(q, k, v, mask, d_model, num_heads,
                         Wq, Wk, Wv, Wo):
    """
    实现 Multi-Head Attention。

    Args:
        q: Query 张量，形状 (batch_size, seq_len_q, d_model)。
        k: Key 张量，形状 (batch_size, seq_len_k, d_model)。
        v: Value 张量，形状 (batch_size, seq_len_v, d_model)。
            seq_len_k 必须等于 seq_len_v。
        mask: Mask 张量 (用于广播到注意力分数)。
        d_model: 模型的维度。
        num_heads: 头数。
        Wq, Wk, Wv, Wo: 线性投影权重矩阵 (d_model, d_model)。

    Returns:
        output: MHA 输出，形状 (batch_size, seq_len_q, d_model)。
        attention_weights: 所有头的注意力权重 (用于可视化或调试)，形状 (batch_size, num_heads, seq_len_q, seq_len_k)。
    """
    depth = d_model // num_heads
    assert d_model % num_heads == 0

    # 1. 线性投影
    q_proj = np.matmul(q, Wq) # (batch_size, seq_len_q, d_model)
    k_proj = np.matmul(k, Wk) # (batch_size, seq_len_k, d_model)
    v_proj = np.matmul(v, Wv) # (batch_size, seq_len_v, d_model)

    # 2. 分割成 num_heads
    q_heads = split_heads(q_proj, num_heads, depth) # (batch_size, num_heads, seq_len_q, depth)
    k_heads = split_heads(k_proj, num_heads, depth) # (batch_size, num_heads, seq_len_k, depth)
    v_heads = split_heads(v_proj, num_heads, depth) # (batch_size, num_heads, seq_len_v, depth)

    # 3. 缩放点积注意力
    attention_output, attention_weights = scaled_dot_product_attention(
        q_heads, k_heads, v_heads, mask)
    # attention_output 形状 (batch_size, num_heads, seq_len_q, depth)
    # attention_weights 形状 (batch_size, num_heads, seq_len_q, seq_len_k)

    # 4. 合并所有头的输出
    output_combined = combine_heads(attention_output) # (batch_size, seq_len_q, d_model)

    # 5. 最终线性投影
    output = np.matmul(output_combined, Wo) # (batch_size, seq_len_q, d_model)

    return output, attention_weights

# --- Test Multi-Head Attention ---
print("--- Test Multi-Head Attention (MHA) ---")
batch_size = 2
seq_len_q = 4
seq_len_k = 5
d_model = 64
num_heads = 8

# Initialize dummy weights (d_model, d_model)
Wq_mha = init_weights(d_model, d_model)
Wk_mha = init_weights(d_model, d_model)
Wv_mha = init_weights(d_model, d_model)
Wo_mha = init_weights(d_model, d_model)

# Simulate input tensors
q_mha = init_weights(batch_size, seq_len_q, d_model)
k_mha = init_weights(batch_size, seq_len_k, d_model)
v_mha = init_weights(batch_size, seq_len_k, d_model)

# Simulate a mask (e.g., padding mask for key/value sequence)
# Let batch item 0 have seq_len 3, batch item 1 have seq_len 4 (out of 5)
dummy_keys_for_mask = np.array([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0]]) # 1s are actual tokens, 0s are padding
mha_mask = create_padding_mask(dummy_keys_for_mask, pad_token=0) # Shape (2, 1, 1, 5)
print(f"MHA Mask shape: {mha_mask.shape}")

mha_output, mha_weights = multi_head_attention(
    q_mha, k_mha, v_mha, mha_mask, d_model, num_heads,
    Wq_mha, Wk_mha, Wv_mha, Wo_mha
)

print(f"MHA Input Q shape: {q_mha.shape}")
print(f"MHA Input K shape: {k_mha.shape}")
print(f"MHA Input V shape: {v_mha.shape}")
print(f"MHA Output shape: {mha_output.shape}") # Expected: (2, 4, 64)
print(f"MHA Weights shape: {mha_weights.shape}") # Expected: (2, 8, 4, 5)
print("-" * 30)

--- Test Multi-Head Attention (MHA) ---
MHA Mask shape: (2, 1, 1, 5)
MHA Input Q shape: (2, 4, 64)
MHA Input K shape: (2, 5, 64)
MHA Input V shape: (2, 5, 64)
MHA Output shape: (2, 4, 64)
MHA Weights shape: (2, 8, 4, 5)
------------------------------


---

### 2. MQA (Multi-Query Attention)

* **全文 (Full Name):** Multi-Query Attention (多查询注意力)
* **解释 (Explanation):** MQA 的核心思想是为了提高推理速度和减少 K/V 缓存的内存占用，让所有的注意力头 *共享* 同一组 Key 和 Value。Query 仍然是每个头独立的。这意味着 K 和 V 只需被计算和存储一次，而不是像 MHA 那样计算和存储 $h$ 次。
* **数学原理 (Mathematical Principle):**
    $$\text{MQA}(Q, K, V) = \text{Concat}(\text{head}_1, ..., \text{head}_h)W^O$$
    其中 $\text{head}_i = \text{Attention}(QW_i^Q, KW^K, VW^V)$。
    $W_i^Q$ 是第 $i$ 个头的 Query 线性投影权重矩阵。$W^K, W^V$ 是 Key 和 Value 的共享线性投影权重矩阵。Q 被投影成 $h$ 个独立的头，每个形状 $(B, L, d_k)$。K 和 V 被投影成单一份，形状为 $(B, L', d_k)$ 和 $(B, L', d_v)$ (通常 $d_k = d_v = d_{model} / h$ 或者某个较小的维度)。在注意力计算时，这单一份 K 和 V 被所有 Q 头共享。
* **算法 (Algorithm Steps):**
    1.  输入 $Q, K, V$ (形状 $(B, L, d_{model})$)。
    2.  对 $Q$ 应用线性变换（投影），然后分割成 $h$ 个头：$Q_{heads} = \text{split_heads}(QW^Q, h, d_k)$。$W^Q$ 形状通常为 $(d_{model}, d_{model})$，然后 reshape。
    3.  对 $K$ 应用 *一次* 线性变换（投影）：$K_{proj} = KW^K$。$W^K$ 形状 $(d_{model}, d_k)$。
    4.  对 $V$ 应用 *一次* 线性变换（投影）：$V_{proj} = VW^V$。$W^V$ 形状 $(d_{model}, d_v)$。
    5.  将 $K_{proj}$ 和 $V_{proj}$ 增加一个维度，使其可以与 $Q_{heads}$ (形状 $(B, h, L, d_k)$) 进行广播兼容，变成形状 $(B, 1, L', d_k)$ 和 $(B, 1, L', d_v)$。
    6.  在每个 Q 头（跨批次和头维度并行）上，使用 *同一份* $K_{proj}$ 和 $V_{proj}$ 执行缩放点积注意力计算。
    7.  将所有头的注意力输出结果在头维度上拼接起来，reshape 回原始形状 $(B, L, h \cdot d_v)$。
    8.  对拼接后的结果应用最终的线性变换（投影）：$Output = \text{Concat}(\text{heads})W^O$。$W^O$ 形状为 $(h \cdot d_v, d_{model})$。
* **Python 代码实现与测试 (Python Code Implementation & Test):**

In [3]:
def multi_query_attention(q, k, v, mask, d_model, num_heads,
              Wq, Wk_mqa, Wv_mqa, Wo_mqa):
    """
    实现 Multi-Query Attention。

    Args:
        q: Query 张量，形状 (batch_size, seq_len_q, d_model)。
        k: Key 张量，形状 (batch_size, seq_len_k, d_model)。
        v: Value 张量，形状 (batch_size, seq_len_v, d_model)。
            seq_len_k 必须等于 seq_len_v。
        mask: Mask 张量 (用于广播到注意力分数)。
        d_model: 模型的维度。
        num_heads: 头数。
        Wq: Query 线性投影权重矩阵 (d_model, d_model)。
        Wk_mqa, Wv_mqa: Key 和 Value 的共享线性投影权重矩阵。
                        形状 (d_model, d_k_mqa) 和 (d_model, d_v_mqa)。
                        通常 d_k_mqa = d_v_mqa = d_model // num_heads。
        Wo_mqa: 最终线性输出投影矩阵 (num_heads * d_v_mqa, d_model)。

    Returns:
        output: MQA 输出，形状 (batch_size, seq_len_q, d_model)。
        attention_weights: 所有头的注意力权重 (用于可视化或调试)，形状 (batch_size, num_heads, seq_len_q, seq_len_k)。
    """
    depth_q = d_model // num_heads # dimension per Q head
    # For MQA, K/V depth is typically the same as depth_q for simplicity
    depth_k_mqa = depth_q
    depth_v_mqa = depth_q
    # Need to verify if Wk_mqa, Wv_mqa shapes match these depths
    assert Wk_mqa.shape == (d_model, depth_k_mqa)
    assert Wv_mqa.shape == (d_model, depth_v_mqa)
    assert Wo_mqa.shape == (num_heads * depth_v_mqa, d_model)
    assert d_model % num_heads == 0

    # 1. 线性投影 Q 并分割成 num_heads
    q_proj = np.matmul(q, Wq) # (batch_size, seq_len_q, d_model)
    q_heads = split_heads(q_proj, num_heads, depth_q) # (batch_size, num_heads, seq_len_q, depth_q)

    # 2. 线性投影 K 和 V (共享)
    k_mqa_proj = np.matmul(k, Wk_mqa) # (batch_size, seq_len_k, depth_k_mqa)
    v_mqa_proj = np.matmul(v, Wv_mqa) # (batch_size, seq_len_v, depth_v_mqa)

    # 3. 增加维度以匹配 Q_heads 的头维度 (用于广播)
    k_mqa_broadcast = k_mqa_proj[:, np.newaxis, :, :] # (batch_size, 1, seq_len_k, depth_k_mqa)
    v_mqa_broadcast = v_mqa_proj[:, np.newaxis, :, :] # (batch_size, 1, seq_len_v, depth_v_mqa)

    # 4. 缩放点积注意力 (Q_heads 将广播 K_mqa_broadcast 和 V_mqa_broadcast)
    attention_output, attention_weights = scaled_dot_product_attention(
        q_heads, k_mqa_broadcast, v_mqa_broadcast, mask)
    # attention_output 形状 (batch_size, num_heads, seq_len_q, depth_v_mqa)
    # attention_weights 形状 (batch_size, num_heads, seq_len_q, seq_len_k)

    # 5. 合并所有头的输出
    output_combined = combine_heads(attention_output) # (batch_size, seq_len_q, num_heads * depth_v_mqa)

    # 6. 最终线性投影
    output = np.matmul(output_combined, Wo_mqa) # (batch_size, seq_len_q, d_model)

    return output, attention_weights

# --- Test Multi-Query Attention (MQA) ---
print("--- Test Multi-Query Attention (MQA) ---")
batch_size = 2
seq_len_q = 4
seq_len_k = 5
d_model = 64
num_heads = 8
depth_per_head = d_model // num_heads # = 8

# Initialize dummy weights
Wq_mqa = init_weights(d_model, d_model) # Q projection is still (d_model, d_model) before splitting
Wk_mqa = init_weights(d_model, depth_per_head) # K projection is (d_model, d_k)
Wv_mqa = init_weights(d_model, depth_per_head) # V projection is (d_model, d_v)
Wo_mqa = init_weights(num_heads * depth_per_head, d_model) # Output projection is (h*d_v, d_model)

# Simulate input tensors (same as MHA test)
q_mqa = init_weights(batch_size, seq_len_q, d_model)
k_mqa_input = init_weights(batch_size, seq_len_k, d_model)
v_mqa_input = init_weights(batch_size, seq_len_k, d_model)

# Use the same mask example
dummy_keys_for_mask = np.array([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0]]) # 1s are actual tokens, 0s are padding
mqa_mask = create_padding_mask(dummy_keys_for_mask, pad_token=0) # Shape (2, 1, 1, 5)
print(f"MQA Mask shape: {mqa_mask.shape}")


mqa_output, mqa_weights = multi_query_attention(
    q_mqa, k_mqa_input, v_mqa_input, mqa_mask, d_model, num_heads,
    Wq_mqa, Wk_mqa, Wv_mqa, Wo_mqa
)

print(f"MQA Input Q shape: {q_mqa.shape}")
print(f"MQA Input K shape: {k_mqa_input.shape}")
print(f"MQA Input V shape: {v_mqa_input.shape}")
print(f"MQA Output shape: {mqa_output.shape}") # Expected: (2, 4, 64)
print(f"MQA Weights shape: {mqa_weights.shape}") # Expected: (2, 8, 4, 5)
print("-" * 30)

--- Test Multi-Query Attention (MQA) ---
MQA Mask shape: (2, 1, 1, 5)
MQA Input Q shape: (2, 4, 64)
MQA Input K shape: (2, 5, 64)
MQA Input V shape: (2, 5, 64)
MQA Output shape: (2, 4, 64)
MQA Weights shape: (2, 8, 4, 5)
------------------------------


---

### 3. GQA (Grouped-Query Attention)

* **全文 (Full Name):** Grouped-Query Attention (分组查询注意力)
* **解释 (Explanation):** GQA 是 MHA 和 MQA 之间的一种折衷。它将 Query 头分成 $G$ 组 ($1 < G < h$)，每组共享 *自己* 的一组 Key 和 Value。这意味着 K 和 V 被计算和存储 $G$ 次（每组一次），而不是 MHA 的 $h$ 次或 MQA 的 1 次。这在效率和模型性能之间提供了平衡。
* **数学原理 (Mathematical Principle):**
    $$\text{GQA}(Q, K, V) = \text{Concat}(\text{group}_1, ..., \text{group}_G)W^O$$
    其中 $\text{group}_j$ 是第 $j$ 组的头的输出拼接，这组包含 $h/G$ 个 Query 头。
    $$\text{head}_i = \text{Attention}(QW_i^Q, KW_j^K, VW_j^V)$$
    $W_i^Q$ 是第 $i$ 个头的 Query 线性投影。$W_j^K, W_j^V$ 是第 $j$ 组共享的 Key 和 Value 线性投影权重矩阵。Q 被投影成 $h$ 个独立的头，形状 $(B, L, d_k)$。K 和 V 被投影成 $G$ 组，每组形状 $(B, L', d_k)$ 和 $(B, L', d_v)$ (通常 $d_k = d_v = d_{model} / h$ 或某个较小的维度，但 K/V 投影输出的总维度是 $G \cdot d_k$)。
* **算法 (Algorithm Steps):**
    1.  输入 $Q, K, V$ (形状 $(B, L, d_{model})$)。
    2.  对 $Q$ 应用线性变换（投影），然后分割成 $h$ 个头：$Q_{heads} = \text{split_heads}(QW^Q, h, d_k)$。$W^Q$ 形状 $(d_{model}, d_{model})$。
    3.  对 $K$ 应用线性变换（投影），然后分割成 $G$ 组：$K_{groups} = \text{reshape_and_transpose}(KW^K, G, d_k)$。$W^K$ 形状 $(d_{model}, G \cdot d_k)$。结果形状 $(B, G, L', d_k)$。
    4.  对 $V$ 应用线性变换（投影），然后分割成 $G$ 组：$V_{groups} = \text{reshape_and_transpose}(VW^V, G, d_v)$。$W^V$ 形状 $(d_{model}, G \cdot d_v)$。结果形状 $(B, G, L', d_v)$。
    5.  为了与 $Q_{heads}$ (形状 $(B, h, L, d_k)$) 进行计算，需要将 $K_{groups}$ 和 $V_{groups}$ (形状 $(B, G, L', d_{k/v})$) 在组维度上进行重复，使其维度变为 $h$。重复次数是 $h/G$。结果形状 $(B, h, L', d_k)$ 和 $(B, h, L', d_v)$。
    6.  执行缩放点积注意力计算。
    7.  将所有头的注意力输出结果在头维度上拼接起来，reshape 回原始形状 $(B, L, h \cdot d_v)$。
    8.  对拼接后的结果应用最终的线性变换（投影）：$Output = \text{Concat}(\text{heads})W^O$。$W^O$ 形状为 $(h \cdot d_v, d_{model})$。
* **Python 代码实现与测试 (Python Code Implementation & Test):**

In [4]:
def grouped_query_attention(q, k, v, mask, d_model, num_heads, num_groups,
                            Wq, Wk_gqa, Wv_gqa, Wo_gqa):
    """
    实现 Grouped-Query Attention。

    Args:
        q: Query 张量，形状 (batch_size, seq_len_q, d_model)。
        k: Key 张量，形状 (batch_size, seq_len_k, d_model)。
        v: Value 张量，形状 (batch_size, seq_len_v, d_model)。
            seq_len_k 必须等于 seq_len_v。
        mask: Mask 张量 (用于广播到注意力分数)。
        d_model: 模型的维度。
        num_heads: Query 头数。
        num_groups: K/V 组数 (1 < num_groups < num_heads)。
        Wq: Query 线性投影权重矩阵 (d_model, d_model)。
        Wk_gqa, Wv_gqa: Key 和 Value 的分组线性投影权重矩阵。
                        形状 (d_model, num_groups * d_k_gqa) 和 (d_model, num_groups * d_v_gqa)。
                        通常 d_k_gqa = d_v_gqa = d_model // num_heads。
        Wo_gqa: 最终线性输出投影矩阵 (num_heads * d_v_gqa, d_model)。

    Returns:
        output: GQA 输出，形状 (batch_size, seq_len_q, d_model)。
        attention_weights: 所有头的注意力权重 (用于可视化或调试)，形状 (batch_size, num_heads, seq_len_q, seq_len_k)。
    """
    depth_q = d_model // num_heads # dimension per Q head
    # For GQA, K/V depth per group is typically the same as depth_q
    depth_k_gqa = depth_q
    depth_v_gqa = depth_q

    assert d_model % num_heads == 0
    assert num_heads % num_groups == 0 # num_heads 必须能被 num_groups 整除
    assert 1 < num_groups < num_heads # 必须介于 MQA 和 MHA 之间

    heads_per_group = num_heads // num_groups

    # Need to verify if Wk_gqa, Wv_gqa shapes match expected
    assert Wk_gqa.shape == (d_model, num_groups * depth_k_gqa)
    assert Wv_gqa.shape == (d_model, num_groups * depth_v_gqa)
    assert Wo_gqa.shape == (num_heads * depth_v_gqa, d_model)


    # 1. 线性投影 Q 并分割成 num_heads
    q_proj = np.matmul(q, Wq) # (batch_size, seq_len_q, d_model)
    q_heads = split_heads(q_proj, num_heads, depth_q) # (batch_size, num_heads, seq_len_q, depth_q)

    # 2. 线性投影 K 并分割成 num_groups
    k_gqa_proj = np.matmul(k, Wk_gqa) # (batch_size, seq_len_k, num_groups * depth_k_gqa)
    # Reshape to (batch, seq_len_k, num_groups, depth_k_gqa) then transpose to (batch, num_groups, seq_len_k, depth_k_gqa)
    k_groups = k_gqa_proj.reshape(k_gqa_proj.shape[0], k_gqa_proj.shape[1], num_groups, depth_k_gqa).transpose(0, 2, 1, 3)


    # 3. 线性投影 V 并分割成 num_groups
    v_gqa_proj = np.matmul(v, Wv_gqa) # (batch_size, seq_len_v, num_groups * depth_v_gqa)
    # Reshape to (batch, seq_len_v, num_groups, depth_v_gqa) then transpose to (batch, num_groups, seq_len_v, depth_v_gqa)
    v_groups = v_gqa_proj.reshape(v_gqa_proj.shape[0], v_gqa_proj.shape[1], num_groups, depth_v_gqa).transpose(0, 2, 1, 3)

    # 4. 将 K_groups 和 V_groups 沿着组维度 (轴 1) 重复以匹配 Query 的头维度
    # (batch, num_groups, seq_len, depth) -> (batch, num_groups * heads_per_group, seq_len, depth) = (batch, num_heads, seq_len, depth)
    k_gqa_broadcast = np.repeat(k_groups, heads_per_group, axis=1)
    v_gqa_broadcast = np.repeat(v_groups, heads_per_group, axis=1)

    # 5. 缩放点积注意力
    attention_output, attention_weights = scaled_dot_product_attention(
        q_heads, k_gqa_broadcast, v_gqa_broadcast, mask)
    # attention_output 形状 (batch_size, num_heads, seq_len_q, depth_v_gqa)
    # attention_weights 形状 (batch_size, num_heads, seq_len_q, seq_len_k)

    # 6. 合并所有头的输出
    output_combined = combine_heads(attention_output) # (batch_size, seq_len_q, num_heads * depth_v_gqa)

    # 7. 最终线性投影
    output = np.matmul(output_combined, Wo_gqa) # (batch_size, seq_len_q, d_model)

    return output, attention_weights

# --- Test Grouped-Query Attention (GQA) ---
print("--- Test Grouped-Query Attention (GQA) ---")
batch_size = 2
seq_len_q = 4
seq_len_k = 5
d_model = 64
num_heads = 8
num_groups = 4 # Must be a divisor of num_heads, and 1 < num_groups < num_heads
depth_per_head = d_model // num_heads # = 8
heads_per_group = num_heads // num_groups # = 2

# Initialize dummy weights
Wq_gqa = init_weights(d_model, d_model) # Q projection is still (d_model, d_model) before splitting
Wk_gqa = init_weights(d_model, num_groups * depth_per_head) # K projection is (d_model, G*d_k)
Wv_gqa = init_weights(d_model, num_groups * depth_per_head) # V projection is (d_model, G*d_v)
Wo_gqa = init_weights(num_heads * depth_per_head, d_model) # Output projection is (h*d_v, d_model)

# Simulate input tensors (same as MHA test)
q_gqa = init_weights(batch_size, seq_len_q, d_model)
k_gqa_input = init_weights(batch_size, seq_len_k, d_model)
v_gqa_input = init_weights(batch_size, seq_len_k, d_model)

# Use the same mask example
dummy_keys_for_mask = np.array([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0]]) # 1s are actual tokens, 0s are padding
gqa_mask = create_padding_mask(dummy_keys_for_mask, pad_token=0) # Shape (2, 1, 1, 5)
print(f"GQA Mask shape: {gqa_mask.shape}")

gqa_output, gqa_weights = grouped_query_attention(
    q_gqa, k_gqa_input, v_gqa_input, gqa_mask, d_model, num_heads, num_groups,
    Wq_gqa, Wk_gqa, Wv_gqa, Wo_gqa
)

print(f"GQA Input Q shape: {q_gqa.shape}")
print(f"GQA Input K shape: {k_gqa_input.shape}")
print(f"GQA Input V shape: {v_gqa_input.shape}")
print(f"GQA Output shape: {gqa_output.shape}") # Expected: (2, 4, 64)
print(f"GQA Weights shape: {gqa_weights.shape}") # Expected: (2, 8, 4, 5)
print("-" * 30)

--- Test Grouped-Query Attention (GQA) ---
GQA Mask shape: (2, 1, 1, 5)
GQA Input Q shape: (2, 4, 64)
GQA Input K shape: (2, 5, 64)
GQA Input V shape: (2, 5, 64)
GQA Output shape: (2, 4, 64)
GQA Weights shape: (2, 8, 4, 5)
------------------------------
